# **End of week 1 exercise**

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

## **1. Get a response from your favorite AI Tutor** 

In [1]:
import os
from openai import OpenAI
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API key looks good so far


In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

openai = OpenAI()

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [4]:
system_prompt = """You are the software engnieer, phd in mathematics, machine learning engnieer, and other topics"""
system_prompt += """
When responding, always use Markdown for formatting. For any code, use well-structured code blocks with syntax highlighting,
For instance:
```python

sample_list = [for i in range(10)]
```
Another example
```javascript
        function displayMessage() {
            alert("Hello, welcome to JavaScript!");
        }

```

Break down explanations into clear, numbered steps for better understanding. 
Highlight important terms using inline code formatting (e.g., `function_name`, `variable`).
Provide examples for any concepts and ensure all examples are concise, clear, and relevant.
Your goal is to create visually appealing, easy-to-read, and informative responses.

"""


In [5]:
def tutor_user_prompt(question):
    # Ensure the question is properly appended to the user prompt.
    user_prompt = (
        "Please carefully explain the following question in a step-by-step manner for clarity:\n\n"
    )
    user_prompt += question
    return user_prompt

In [6]:


def askTutor(question, MODEL):
    # Generate the user prompt dynamically.
    user_prompt = tutor_user_prompt(question)
    
    # OpenAI API call to generate response.
    if MODEL == 'gpt-4o-mini':
        print(f'You are getting response from {MODEL}')
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=True
        )
    else:
        MODEL == 'llama3.2'
        print(f'You are getting response from {MODEL}')
        stream = ollama_via_openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=True
        )

    # Initialize variables for response processing.
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    # Process the response stream and update display dynamically.
    for chunk in stream:
        # Safely access the content attribute.
        response_chunk = getattr(chunk.choices[0].delta, "content", "")
        if response_chunk:  # Check if response_chunk is not None or empty
            response += response_chunk
            # No replacement of Markdown formatting here!
            update_display(Markdown(response), display_id=display_handle.display_id)


In [7]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [8]:
askTutor(question=question, MODEL=MODEL_GPT)

You are getting response from gpt-4o-mini


Certainly! Let's break down the provided code step-by-step for clarity:

### Code Overview
```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Step-by-Step Explanation

1. **Understanding `yield from`**:
   - The keyword `yield from` allows a generator to yield all values from another iterable. It’s a way to delegate generation to another generator or iterable.
   - In this context, `yield from` indicates that this line of code is part of a generator function.

2. **Understanding the Set Comprehension**:
   - The code `{book.get("author") for book in books if book.get("author")}` is a **set comprehension**.
   - A set comprehension constructs a set (a collection of unique elements) by iterating over an iterable (in this case, `books`), applying a certain condition.

3. **Iterating Over `books`**:
   - `books` is expected to be a collection (like a list) of dictionaries, where each dictionary represents a book with various attributes, including the author's name.
   - This iterable is processed one book (dictionary) at a time.

4. **Using `book.get("author")`**:
   - The method `get()` is called on each `book` dictionary to retrieve the value associated with the key `"author"`.
   - If the key does not exist, `book.get("author")` returns `None` instead of raising a KeyError.

5. **Filtering with the Condition**:
   - The condition `if book.get("author")` ensures that only books with a defined (non-`None`) author are processed. 
   - This means books without an author will be ignored in the set comprehension.

6. **Creating the Set**:
   - The set comprehension builds a set of authors for the books that have non-`None` authors. Since sets automatically discard duplicate entries, each author will only appear once in the resulting set.

7. **Yielding from the Set**:
   - Finally, `yield from` will yield each author's name from the set, one by one, to whoever is iterating over the generator function.

### Summary
- The code effectively extracts unique author names from a collection of book dictionaries and yields them one by one.
- This is particularly useful for ensuring that duplicate author names do not appear, resulting in a cleaner output.

### Example Usage
Here’s a minimal example demonstrating how this fits into a generator function:

```python
def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example data
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": None},
    {"title": "Book Four", "author": "Author A"},
]

# Iterating over the generator
for author in unique_authors(books):
    print(author)
```

### Output
```
Author A
Author B
```

This example will print unique authors `Author A` and `Author B`, demonstrating the functionality of the original code.

## **2. Using both LLMs collaboratively approach**

- I thought about like similar the idea of a RAG (Retrieval-Augmented Generation) approach, is an excellent idea to improve responses by refining the user query and producing a polished, detailed final answer. Two LLM talking each other its cool!!! Here's how we can implement this:

**Updated Concept:**
1. Refine Query with Ollama:
    - Use Ollama to refine the raw user query into a well-structured prompt.
    - This is especially helpful when users input vague or poorly structured queries.
2. Generate Final Response with GPT:
    - Pass the refined prompt from Ollama to GPT to generate the final, detailed, and polished response.
3. Return the Combined Output:
    - Combine the input, refined query, and the final response into a single display to ensure clarity.

In [9]:
def refine_with_ollama(raw_question):
    """
    Use Ollama to refine the user's raw question into a well-structured prompt.
    """
    print("Refining the query using Ollama...")
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Refine and structure the following user input."},

        {"role": "user", "content": raw_question},
    ]
    response = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=messages,
        stream=False  # Non-streamed refinement
    )
    refined_query = response.choices[0].message.content
    return refined_query

In [10]:
def ask_with_ollama_and_gpt(raw_question):
    """
    Use Ollama to refine the user query and GPT to generate the final response.
    """
    # Step 1: Refine the query using Ollama
    refined_query = refine_with_ollama(raw_question)
    
    # Step 2: Generate final response with GPT
    print("Generating the final response using GPT...")
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": refined_query},
    ]
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True  # Stream response for dynamic display
    )

    # Step 3: Combine responses
    response = ""
    display_handle = display(Markdown(f"### Refined Query:\n\n{refined_query}\n\n---\n\n### Final Response:"), display_id=True)
    for chunk in stream:
        response_chunk = getattr(chunk.choices[0].delta, "content", "")
        if response_chunk:
            response += response_chunk
            update_display(Markdown(f"### Refined Query:\n\n{refined_query}\n\n---\n\n### Final Response:\n\n{response}"), display_id=display_handle.display_id)

In [11]:
# Example Usage
question = """
Please explain what this code does:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [12]:
ask_with_ollama_and_gpt(raw_question=question)

Refining the query using Ollama...
Generating the final response using GPT...


### Refined Query:

Here is a refined version of your request:

**Ex Explanation**

This line of Python code is using a feature called Generators to fetch all the authors from a list of books.

```python
from typing import Generator

def extract_authors(books: dict) -> Generator:
    """
    Yields a generator that contains only the authors from the provided list of books.
    
    Args:
        books (dict): A dictionary where each key-value pair represents a book, 
                      with 'author' as its value.
    """

    for book in books.values():
        yield book.get("author")

# Usage
books = {
    "Book1": {"title": "1984", "author": "George Orwell"},
    "Book2": {"title": "Dune", "author": None},
    "Book3": {"title": "The Lord of the Rings", "author": "J.R.R. Tolkien"}
}

for author in extract_authors(books):
    print(author)
```

**Output**
```
George Orwell
None
J.R.R. Tolkien
```

This code works by iterating through each item `book` in the dictionary of books, checking if it has an author, and yielding that author's name.

It appears to be an attempt to implement a basic pagination or results filtering mechanism using Generators however due to missing imports (like `typing`,), further adjustments may need to made depending on use case requirements. 

However, there is still an issue here with `"author": None` in `"Book2"` - so if book is supposed to be able to have author and possibly none then we would want change it from `get()` and set the default of what value should be used when getting a field that may potentially not exist.

---

### Final Response:

Certainly! Let’s break down the provided Python code step by step, along with addressing a couple of refinement points as highlighted in your explanation.

### Step-by-Step Explanation

1. **Import the Required Module**:
   The code begins by importing `Generator` from the `typing` module, which is useful for type hinting and indicating that the function will yield values instead of returning a list.

    ```python
    from typing import Generator
    ```

2. **Define the Function**:
   The function `extract_authors` is defined which accepts a dictionary named `books`. This dictionary contains key-value pairs representing books, where each value is another dictionary with details about the book.

    ```python
    def extract_authors(books: dict) -> Generator:
    ```

3. **Docstring**:
   A docstring explains what the function does, what arguments it receives, and provides a brief description of the data structure. This is essential for code readability and maintainability.

   ```python
    """
    Yields a generator that contains only the authors from the provided list of books.
    
    Args:
        books (dict): A dictionary where each key-value pair represents a book, 
                      with 'author' as its value.
    """
    ```

4. **Iterate Through Books**:
   The function uses a loop to iterate through each `book` in the values of the `books` dictionary.

    ```python
    for book in books.values():
    ```

5. **Yield Author Names**:
   The `yield` statement retrieves the author's name using `book.get("author")` and yields it. If the `author` key doesn't exist, `None` will be returned by default.

    ```python
        yield book.get("author")
    ```

6. **Usage Example**:
   A sample dictionary `books` is defined containing three books, two of which have specified authors while one has an author set to `None`.

   ```python
   books = {
       "Book1": {"title": "1984", "author": "George Orwell"},
       "Book2": {"title": "Dune", "author": None},
       "Book3": {"title": "The Lord of the Rings", "author": "J.R.R. Tolkien"}
   }
   ```

7. **Output**:
   The authors are printed using a for loop that calls the `extract_authors` function. The output displays each author's name, including `None` for "Book2".

   ```python
   for author in extract_authors(books):
       print(author)
   ```

### Sample Output
The output from running the code will be:

```
George Orwell
None
J.R.R. Tolkien
```

### Refining the Code

To address the potential issue with `None` for the author, a more explicit handling might be desired. Below is an improved version of the function that allows you to set a default value for missing authors.

#### Code Refinement
```python
from typing import Generator, Optional

def extract_authors(books: dict, default: Optional[str] = "Unknown Author") -> Generator:
    """
    Yields a generator containing authors from the provided list of books, 
    defaulting to a specific value if the author is None or missing.
    
    Args:
        books (dict): A dictionary with each key-value representing a book.
        default (str): The default value to use for books without an author.
    """

    for book in books.values():
        yield book.get("author", default)

# Usage remains the same.
for author in extract_authors(books):
    print(author)
```

### Changes Explained:
- **Parameter `default`**: This parameter allows the user to specify a default value for authors that are not present or are set to `None`.
- **Using `book.get("author", default)`**: This ensures that if an author's name is not found, the provided default value is returned instead.

### Final Output with Refined Code
Executing the refined code will now yield:

```
George Orwell
Unknown Author
J.R.R. Tolkien
```

This introduces clarity and handles edge cases more gracefully while maintaining code readability.